In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import fftconvolve
from scipy.optimize import curve_fit

# パラメータ設定
sigma_model = 5 / 2.35  # スペクトルモデルの標準偏差 (eV)
sigma_detector = 4.5 / 2.35  # 検出器応答の標準偏差 (eV)
energy_range = np.linspace(-100, 100, 1000)  # エネルギー範囲 (eV)
total_events = 1e5  # 観測中の総事象数

# 観測時間の設定 (ksec単位)
observation_times = np.linspace(0, 10, 10)  # 観測時間を0~10 ksecで10個

# 観測時間に応じた中心エネルギーの計算
def calculate_center_energy(observation_time):
    return 0.124 * observation_time  # 中心エネルギーが 0.124 eV/ksec で変化

# ガウス関数の定義
def gaussian(x, a, x0, sigma):
    return a * np.exp(-(x - x0)**2 / (2 * sigma**2))

# 検出器応答 (ガウス分布, 観測時間に依存しない)
detector_response = np.exp(-energy_range**2 / (2 * sigma_detector**2))
detector_response /= np.sum(detector_response)  # 正規化

# スペクトルの足し合わせ
total_spectrum = np.zeros_like(energy_range)
for observation_time in observation_times:
    center_model = calculate_center_energy(observation_time)  # 観測時間ごとの中心エネルギー

    # スペクトルモデル (ガウス分布)
    model_spectrum = np.exp(-(energy_range - center_model)**2 / (2 * sigma_model**2))
    model_spectrum /= np.sum(model_spectrum)  # 正規化

    # 畳み込みして観測スペクトルを計算
    observed_spectrum = fftconvolve(model_spectrum, detector_response, mode='same')

    # 観測時間を考慮してイベント数を生成
    observed_spectrum *= total_events / np.sum(observed_spectrum)

    # 全体スペクトルに足し合わせ
    total_spectrum += observed_spectrum

# フィッティング
popt, pcov = curve_fit(gaussian, energy_range, total_spectrum, p0=[total_spectrum.max(), 0, 10])

# フィッティング結果
fitted_a, fitted_x0, fitted_sigma = popt
print(f"Fitted Parameters:\n  Amplitude: {fitted_a:.2f}\n  Center: {fitted_x0:.2f} eV\n  Sigma: {fitted_sigma:.2f} eV")

# プロット
plt.figure(figsize=(10, 7))
plt.plot(energy_range, total_spectrum, label="Summed Spectrum", color="blue")
plt.plot(energy_range, gaussian(energy_range, *popt), label="Fitted Gaussian", color="red", linestyle="--")
plt.xlabel("Energy (eV)")
plt.ylabel("Counts")
plt.title("Summed Spectrum and Gaussian Fit")
plt.legend()
plt.grid(True)
plt.show()

: 